# Topic Modeling

## Import

In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import matplotlib.dates

import seaborn as sns

from src.progress_bar import printProgressBar

from ast import literal_eval

import gensim
import gensim.corpora as corpora
from gensim.models import TfidfModel

import pyLDAvis
import pyLDAvis.gensim_models

import spacy

import json

import os
for dirname, _, filenames in os.walk('../../data/lyrics'):
    for filename in filenames:
        print(os.path.join(dirname, filename).replace("\\", "/"))

print(50 * '-')

for dirname, _, filenames in os.walk('../../data/billboard'):
    for filename in filenames:
        print(os.path.join(dirname, filename).replace("\\", "/"))

../../data/lyrics/artist_song_lyrics.csv
../../data/lyrics/artist_song_lyrics_new.csv
../../data/lyrics/bb-t100-lyrics.csv
../../data/lyrics/bb-t100-lyrics_new.csv
../../data/lyrics/bb_t100_lyrics_en.csv
../../data/lyrics/bb_t100_lyrics_en_new.csv
../../data/lyrics/covid_lyrics_bb.csv
../../data/lyrics/lyrics_invalid.json
../../data/lyrics/lyrics_invalid_new.json
../../data/lyrics/lyrics_invalid_updated.csv
../../data/lyrics/lyrics_invalid_updated_new.csv
../../data/lyrics/backups/artist_song_lyrics_bak.csv
../../data/lyrics/backups/bb-t100-lyrics.csv
../../data/lyrics/backups/bb-t100-lyrics_new.csv
../../data/lyrics/backups/bb-t100-lyrics_old.csv
../../data/lyrics/backups/bb_t100_lyrics_en.csv
../../data/lyrics/backups/bb_t100_lyrics_en_v1.csv
../../data/lyrics/backups/bb_t100_lyrics_en_v2.csv
../../data/lyrics/backups/bb_t100_lyrics_en_v3.csv
../../data/lyrics/backups/corona-lyrics.csv
../../data/lyrics/backups/lyrics_invalid_updated.csv
----------------------------------------------

## Loading Data

### Lyrics

In [2]:
lyrics = pd.read_csv('../../data/lyrics/bb_t100_lyrics_en.csv', index_col=0, encoding='utf-8')
lyrics['first_appearance'] = pd.to_datetime(lyrics['first_appearance'], format='%Y-%m-%d')
lyrics['release_date'] = pd.to_datetime(lyrics['release_date'], format='%Y-%m-%d')
lyrics['genius_annotations'] = lyrics['genius_annotations'].apply(literal_eval)
lyrics['genius_comments'] = lyrics['genius_comments'].apply(literal_eval)
lyrics.reset_index(inplace=True)
lyrics.head()

,index,billboard_id,lyrics_id,artist,first_artist,song,weeks_on_chart,peak_rank,genius_id,lyrics,...,word_count,language,language_score,first_appearance,genius_primary_artist,genius_description,genius_annotations,genius_comments,release_date,annotation_ids
0,0,0,0,Ariana Grande,Ariana Grande,"Thank U, Next",28.0,1.0,4063065,Thought I'd end up with Sean\nBut he wasn't a ...,...,460,en,0.999997,2019-01-05,Ariana Grande,On the lead single and titular track to her fi...,[(One taught me love\n One taught me patience\...,"[The Mac shoutout has me fully in tears, this ...",2018-11-03,"['15720075', '15720076', '15720054', '15720247..."
1,1,1,1,Halsey,Halsey,Without Me,52.0,1.0,3977187,Found you when your heart was broke\nI filled ...,...,435,en,0.999995,2019-01-05,Halsey,“Without Me” is the first new song released by...,[(Gave love ’bout a hundred tries (Hundred tri...,[The queen is ready to snatch our wigs once ag...,2018-10-04,"['15517989', '15520369', '15518283', '15518820..."
2,2,2,2,Mariah Carey,Mariah Carey,All I Want For Christmas Is You,43.0,1.0,204233,I don't want a lot for Christmas\nThere is jus...,...,388,en,0.999996,2019-01-05,Mariah Carey,“All I Want For Christmas Is You” is an uptemp...,[(I don’t need to hang my stocking\n There upo...,"[i really like this song, it’s about that time...",1994-11-01,"['8393500', '8393500', '21611023', '8393500', ..."
3,3,3,3,Travis Scott,Travis Scott,Sicko Mode,52.0,1.0,3876994,"Astro, yeah\nSun is down, freezin' cold\nThat'...",...,771,en,0.999998,2019-01-05,Travis Scott,“SICKO MODE” refers to Travis and Drake’s work...,"[(She’s in love with who I am, [['Since Drake ...",[HAD ME OUT LIKE A LIGHT (ayy) LIKE A LIGHT (y...,2018-08-03,"['15114078', '17948214', '15113868', '15113778..."
4,4,4,4,Post Malone & Swae Lee,Post Malone,Sunflower (Spider-Man: Into The Spider-Verse),53.0,1.0,3993850,"Ayy, ayy, ayy, ayy (Ooh)\nOoh, ooh, ooh, ooh (...",...,305,en,0.999997,2019-01-05,Post Malone & Swae Lee,“Sunflower” marks the second collaboration by ...,"[(Then you’re left in the dust, unless I stuck...",[Me enjoying “Sunflower” and someone then inte...,2018-10-18,"['16057378', '16057378']"


### AZLyrics

In [3]:
import csv
import glob


def _get_csv_file_list(unzipping_output_folder):
    """
    Extract all the csv file paths given the folder path.
    :param unzipping_output_folder: Folder path.
    :return: List of CSV file paths.
    """
    csv_file_list = [i for i in glob.glob(f'{unzipping_output_folder}/**/*.csv')]
    print(f'{len(csv_file_list)} CSV files extracted.')
    return csv_file_list

azlyrics = []
def _read(csv_file_list):
    """
    Read the list of data CSVs.
    :param csv_file_list: List of CSV file paths where the data is.
    :return: CSVs read.
    """
    total = 0
    for csv_file_name in csv_file_list:
        with open(csv_file_name, 'r') as csv_file:
            rows = [row for row in csv.reader(csv_file) if row][1:]
            total += len(rows)
            print(f'{csv_file_name}: {len(rows)} songs.')
        azlyrics.append(rows)
    print(f'Total: {total}')


data_folder = '../../data/input/'
csv_file_list = _get_csv_file_list(data_folder)
_read(csv_file_list)

27 CSV files extracted.
../../data/input\azlyrics-scraper\azlyrics_lyrics_19.csv: 868 songs.
../../data/input\azlyrics-scraper\azlyrics_lyrics_a.csv: 13046 songs.
../../data/input\azlyrics-scraper\azlyrics_lyrics_b.csv: 22672 songs.
../../data/input\azlyrics-scraper\azlyrics_lyrics_c.csv: 10296 songs.
../../data/input\azlyrics-scraper\azlyrics_lyrics_d.csv: 6349 songs.
../../data/input\azlyrics-scraper\azlyrics_lyrics_e.csv: 5414 songs.
../../data/input\azlyrics-scraper\azlyrics_lyrics_f.csv: 2450 songs.
../../data/input\azlyrics-scraper\azlyrics_lyrics_g.csv: 4542 songs.
../../data/input\azlyrics-scraper\azlyrics_lyrics_h.csv: 4258 songs.
../../data/input\azlyrics-scraper\azlyrics_lyrics_i.csv: 2161 songs.
../../data/input\azlyrics-scraper\azlyrics_lyrics_j.csv: 11975 songs.
../../data/input\azlyrics-scraper\azlyrics_lyrics_k.csv: 4610 songs.
../../data/input\azlyrics-scraper\azlyrics_lyrics_l.csv: 6151 songs.
../../data/input\azlyrics-scraper\azlyrics_lyrics_m.csv: 12239 songs.
../..

In [4]:
azlyrics_data = {
    'artists' : list(),
    'artist_links' : list(),
    'songs' : list(),
    'song_links' : list(),
    'lyrics' : list()
}
for doc in azlyrics:
    for row in doc:
        if len(row) == 5:
            azlyrics_data['artists'].append(row[0])
            azlyrics_data['artist_links'].append(row[1])
            azlyrics_data['songs'].append(row[2])
            azlyrics_data['song_links'].append(row[3])
            azlyrics_data['lyrics'].append(row[4])

azlyrics_df = pd.DataFrame(data=azlyrics_data)
azlyrics_df.head()

,artists,artist_links,songs,song_links,lyrics
0,03 greedo,https://www.azlyrics.com/19/03greedo.html,sweet lady,https://www.azlyrics.com/lyrics/03greedo/sweet...,"its only one, 03, i'm from grape street, where..."
1,03 greedo,https://www.azlyrics.com/19/03greedo.html,mafia business,https://www.azlyrics.com/lyrics/03greedo/mafia...,"you gonna make me put you in a suit and tie, s..."
2,03 greedo,https://www.azlyrics.com/19/03greedo.html,paranoid,https://www.azlyrics.com/lyrics/03greedo/paran...,"we could kill it, yeah, we could, we could, we..."
3,03 greedo,https://www.azlyrics.com/19/03greedo.html,never bend,https://www.azlyrics.com/lyrics/03greedo/never...,"yeah, you lil bitch ass niggas steady speaking..."
4,03 greedo,https://www.azlyrics.com/19/03greedo.html,prayer for my lost,https://www.azlyrics.com/lyrics/03greedo/praye...,"x loaded up bro, never see me in the church, p..."


## Tokenization

### Lyrics

In [5]:
nlp = spacy.load("en_core_web_lg")
nlp.Defaults.stop_words |= {'ai', 'gon', '\u2005', 'ooh', 'let', 've', 'gonna', 'woah', 'gotta', 'll', 'mmm', 'maybe',
                            'got', 'oh', 'uh', 'want', 'huh', 'ah', 'ma', 'hey', 'woo', 'ain', 'come', 'goin', 'ya',
                            'la', 'bah', 'yeah', 'ayy', 'em', 'tryna', 'goes', 'damn', 'ooo', 'comin', 'ran', 'wanna',
                            'okay', 'til', 'didn', "i\'mma", "mi", "da", "dem", "gyal", "yuh", "fi", "di", "nuh", "ni", "yo", "dum", "gal" "wi", "w", "inna"}

In [6]:
def create_doc(text, pos_tags=["NOUN", 'PROPN', "VERB", "ADJ"]):
    lines = text.split('\n')
    lines_unique_list = []
    for line in lines:
        if len(line) == 0:
            continue
        if line not in lines_unique_list:
            lines_unique_list.append(line)
    unique_lines = " ".join(lines_unique_list)

    lemmas = []
    doc = nlp(unique_lines)
    for token in doc:
        if token.is_stop or token.is_punct:
            continue

        if token.pos_ in pos_tags:
            if token.lemma_ not in nlp.Defaults.stop_words:
                lemmas.append(token.lemma_.lower())
            else:
                continue

    return lemmas


In [7]:
try:
    with open("../../data/cache/topic_docs.txt", "r") as f:
        docs = json.load(f)
    f.close()
except FileNotFoundError:
    docs = []
    i = 0
    l = len(lyrics['lyrics'])
    printProgressBar(i, l, prefix='Progress:', suffix='Complete', length=50)
    for text in lyrics['lyrics']:
        docs.append(create_doc(text))
        i+=1
        printProgressBar(i, l, prefix='Progress:', suffix='Complete', length=50)

    with open("../../data/cache/topic_docs.txt", "w") as f:
        json.dump(docs, f)
    f.close()


### AZLyrics

In [8]:
from spacy.language import Language
from langdetect import DetectorFactory
from spacy_langdetect import LanguageDetector

@Language.factory("language_detector")
def create_language_detector(nlp, name):
    return LanguageDetector(language_detection_function=None)

nlp.add_pipe('language_detector')

In [9]:
def create_doc(text, pos_tags=["NOUN", 'PROPN', "VERB", "ADJ"]):
    lemmas = []
    doc = nlp(text)
    if doc._.language['language'] == 'en' and doc._.language['score'] > 0.95:
        for token in doc:
            if token.is_stop or token.is_punct:
                continue

            if token.pos_ in pos_tags:
                if token.lemma_ not in nlp.Defaults.stop_words:
                    lemmas.append(token.lemma_.lower())
                else:
                    continue
    return lemmas

In [10]:
import random

try:
    with open("../../data/cache/azlyrics_docs_test.txt", "r") as f:
        azlyrics_docs = json.load(f)
    f.close()
except FileNotFoundError:
    random_lyrics = azlyrics_data['lyrics']
    random.shuffle(random_lyrics)
    texts = random_lyrics[:500]
    # texts = azlyrics_data['lyrics'] #very time consuming
    azlyrics_docs = []
    i = 0
    l = len(texts)
    printProgressBar(0, l, prefix='Progress:', suffix='Complete', length=50)
    for text in texts:
        azlyrics_docs.append(create_doc(text))
        i+=1
        printProgressBar(i, l, prefix='Progress:', suffix='Complete', length=50)
    with open("../../data/cache/azlyrics_docs_test.txt", "w") as f:
        json.dump(azlyrics_docs, f)
    f.close()

Progress: |██████████████████████████████████████████████████| 100.0% Complete


## Generate Corpus and Id2Word-Dictionary

### Lyrics

In [11]:
def optimize_corpus(corpus, id2word, words_missing_in_tfidf = [], low_value= 0.03):
    tfidf = TfidfModel(corpus, id2word=id2word)
    dropped_words = []
    l = len(corpus)
    for i in range(0, l):
        bow = corpus[i]
        tfidf_ids = [tfidf_id for tfidf_id, tfidf_value in tfidf[bow]]
        bow_ids = [bow_id for bow_id, bow_value in bow]
        low_value_words = [tfidf_id for tfidf_id, tfidf_value in tfidf[bow] if tfidf_value < low_value]
        drops = low_value_words+words_missing_in_tfidf
        for word_id in drops:
            dropped_words.append(id2word[word_id])
        words_missing_in_tfidf = [bow_id for bow_id in bow_ids if bow_id not in tfidf_ids]
        new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
        corpus[i] = new_bow
        printProgressBar(i+1, l, prefix='Progress:', suffix='Complete', length=50)
    return corpus, dropped_words, words_missing_in_tfidf

In [12]:
texts = docs
id2word = corpora.Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]

corpus = optimize_corpus(corpus, id2word)[0]

Progress: |██████████████████████████████████████████████████| 100.0% Complete


### AZLyrics

In [13]:
azlyrics_texts = azlyrics_docs
azlyrics_id2word = corpora.Dictionary(texts)
azlyrics_corpus = [azlyrics_id2word.doc2bow(text) for text in azlyrics_texts]

azlyrics_corpus = optimize_corpus(azlyrics_corpus, azlyrics_id2word)[0]

Progress: |██████████████████████████████████████████████████| 100.0% Complete


## Modeling

### LDA Model for Billboard Lyrics (don't use!)

In [15]:
# lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
#                                             id2word=id2word,
#                                             num_topics=10,
#                                             update_every=1,
#                                             chunksize=100,
#                                             passes=10,
#                                             alpha='auto',
#                                             random_state=1)

Topic Visualization

In [16]:
# import pyLDAvis
# import pyLDAvis.gensim_models
#
# pyLDAvis.enable_notebook()
# pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds")

### LDA Model for AZLyrics

In [14]:
try:
    azlyrics_lda_model = gensim.models.ldamodel.LdaModel.load('../../gensim/models/azlyrics_lda_test')
except FileNotFoundError:
    azlyrics_lda_model = gensim.models.ldamodel.LdaModel(corpus=azlyrics_corpus,
                                                id2word=azlyrics_id2word,
                                                num_topics=7,
                                                chunksize=2000,
                                                passes=20,
                                                alpha='auto',
                                                iterations=200,
                                                random_state=1)
    azlyrics_lda_model.save('../../gensim/models/azlyrics_lda_test')


Topic Visualization

In [15]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(azlyrics_lda_model, azlyrics_corpus, azlyrics_id2word)

c:\users\fried\documents\1_bildung\2_programmierung\python\datascience\chart-lyrics-analysis\venv\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.242586  0.054555       1        1  57.319146
0     -0.066592 -0.125254       2        1  17.528362
6      0.029278 -0.058757       3        1   7.070536
3      0.005180  0.061292       4        1   5.351900
1      0.113667  0.029402       5        1   4.931480
2      0.083825 -0.005996       6        1   4.020843
4      0.077229  0.044759       7        1   3.777733, topic_info=          Term        Freq       Total Category  logprob  loglift
30        love  508.000000  508.000000  Default  30.0000  30.0000
69        feel  266.000000  266.000000  Default  29.0000  29.0000
991  beautiful   49.000000   49.000000  Default  28.0000  28.0000
53        time  273.000000  273.000000  Default  27.0000  27.0000
498      nigga  113.000000  113.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
151        boy    2.732283   30.835797   Topic7  -6.0535   0.8525
53        time    3.335185  273.451613   Topic7  -5.8541  -1.1306
69        feel    3.305828  266.138756   Topic7  -5.8630  -1.1123
16        good    2.859867  116.340047   Topic7  -6.0079  -0.4297
246       play    2.659686   64.604077   Topic7  -6.0805   0.0860

[456 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
3789       2  0.135607       able
3789       5  0.881448       able
3220       3  0.819898      adams
3287       1  0.968514  addiction
60         1  0.985663     afraid
...      ...       ...        ...
1473       2  0.193558     yellow
1473       5  0.709713     yellow
1473       6  0.064519     yellow
13032      4  0.527445       yore
1508       2  0.809972      youth

[842 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 1, 7, 4, 2, 3, 5])

## Calculate Topics for Lyrics

Topic interpreatation based on LDA visualisation.

In [20]:
topic_names = {
    0 : 'Love',
    1 : 'Crime',
    2 : 'Crime / Party',
    3 : 'Christmas',
    4 : '',
    5 : '',
    6 : '',
    7 : '',
    8 : '',
    9 : '',
}

In [21]:
def get_topics(index, lda_corpus = lda_model[corpus]):
    topics = {}
    for lda_tuple in lda_corpus[index]:
        topics[lda_tuple[0]] = lda_tuple[1]
    return dict(sorted(topics.items(), key=lambda item: item[1], reverse=True))

NameError: name 'lda_model' is not defined

In [ ]:
def get_topics_str(topics):
    return [topic_names[topic] for topic in topics.keys()]

In [ ]:
lyrics['topics'] = [get_topics(i) for i in range(0, len(lyrics))]
lyrics['top_topic'] = lyrics['topics'].apply(lambda topic: list(topic.keys())[0])
# lyrics['topics_str'] = lyrics['topics'].apply(lambda topics: [topic_names[topic] for topic in topics.keys()])
# lyrics['top_topic_str'] = lyrics['top_topic'].apply(lambda topic: topic_names[topic])
# lyrics[['topics', 'topics_str', 'top_topic', 'top_topic_str']]
lyrics[['topics', 'top_topic']]

In [ ]:
# lyrics_topics = lyrics[
#     ['billboard_id', 'lyrics_id', 'artist', 'song', 'weeks_on_chart', 'peak_rank', 'lyrics', 'url', 'first_appearance',
#      'release_date', 'topics', 'top_topic', 'top_topic_str', 'topics_str']]
lyrics_topics = lyrics[
    ['billboard_id', 'lyrics_id', 'artist', 'song', 'weeks_on_chart', 'peak_rank', 'lyrics', 'url', 'first_appearance',
     'release_date', 'topics', 'top_topic']]
lyrics_topics.to_excel('../../data/topic_modeling/bb_t100_sentiment.xlsx')